In [325]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [326]:
artists = pd.read_csv('../data/artist_cleaned.csv')
songs = pd.read_csv('../data/songs_cleaned.csv')

In [327]:
songs.shape

(937, 7)

In [328]:
artists.head()

,artist,description,images_url,followers_count,gender,birth_year,birth_decade
0,Post Malone,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s
1,The Weeknd,"Abel Makkonen Tesfaye (born February 16, 1990 ...",https://images.genius.com/f0813e600d43b8b43c94...,8354.0,Male,1990.0,1990s
2,Roddy Ricch,"Rodrick Wayne Moore, Jr. (b. October 22, 1998)...",https://images.genius.com/78cf8142bcc24004ee49...,749.0,Male,1998.0,1990s
3,Rod Wave,"Rodarius Marcell Green (born August 27, 1998),...",https://images.genius.com/4a800d74ab2eb5821156...,262.0,Male,1998.0,1990s
4,Doja Cat,Amala Ratna Zandile Dlamini (born on October 2...,https://images.genius.com/2686a106bac2a8a7c1e9...,1151.0,Female,1995.0,1990s


In [352]:
songs.head()

,song_name,artist,lyrics,descriptionsong,accepted_annotations,contributors,pageviews,descriptionartist,images_url,followers_count,gender,birth_year,birth_decade,cleaned_lyrics
0,​​rockstar,Post Malone,"[Intro: Post Malone] Hahahahaha Tank God Ayy, ...","On “rockstar,” Post Malone compares his habits...",18.0,440.0,6883347.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro: post malone] hahahahaha tank god ayy a...
1,White Iverson,Post Malone,[Intro] Double OT I'm a new three [Chorus] Sau...,"Virtually unknown before this track, Post Malo...",31.0,222.0,4144447.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro] double ot im a new three [chorus] sauc...
2,Congratulations,Post Malone,"[Intro: Post Malone] Mm-mmm Yeah, yeah Mm-mmm ...","On “Congratulations,“ Post Malone and Quavo ce...",13.0,221.0,3633318.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro: post malone] mm_mmm mm_mmm yeah (hey) ...
3,Psycho,Post Malone,"[Chorus: Post Malone] Damn, my AP goin' psycho...",“Psycho” is the third single from Post Malone’...,21.0,230.0,3080338.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[chorus: post malone] damn my ap going psycho ...
4,I Fall Apart,Post Malone,"[Intro] Ooh, I fall apart Ooh, yeah, mmm, yeah...","On “I Fall Apart,” Post reminisces about a hea...",7.0,126.0,2640350.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro] ooh i fall apart ooh yeah mmm yeah [v...


# Dropping Songs Not in English

In [353]:
songs = songs[songs['artist'] != 'Bad Bunny']
songs = songs[songs['artist'] != 'BTS']

# Merging Songs and Artists to get artist information

In [354]:
songs = songs.merge(artists, how = 'left', on = 'artist', suffixes = ('song','artist'))

In [355]:
songs.head()

,song_name,artist,lyrics,descriptionsong,accepted_annotations,contributors,pageviews,descriptionartist,images_urlsong,followers_countsong,gendersong,birth_yearsong,birth_decadesong,cleaned_lyrics,description,images_urlartist,followers_countartist,genderartist,birth_yearartist,birth_decadeartist
0,​​rockstar,Post Malone,"[Intro: Post Malone] Hahahahaha Tank God Ayy, ...","On “rockstar,” Post Malone compares his habits...",18.0,440.0,6883347.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro: post malone] hahahahaha tank god ayy a...,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s
1,White Iverson,Post Malone,[Intro] Double OT I'm a new three [Chorus] Sau...,"Virtually unknown before this track, Post Malo...",31.0,222.0,4144447.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro] double ot im a new three [chorus] sauc...,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s
2,Congratulations,Post Malone,"[Intro: Post Malone] Mm-mmm Yeah, yeah Mm-mmm ...","On “Congratulations,“ Post Malone and Quavo ce...",13.0,221.0,3633318.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro: post malone] mm_mmm mm_mmm yeah (hey) ...,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s
3,Psycho,Post Malone,"[Chorus: Post Malone] Damn, my AP goin' psycho...",“Psycho” is the third single from Post Malone’...,21.0,230.0,3080338.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[chorus: post malone] damn my ap going psycho ...,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s
4,I Fall Apart,Post Malone,"[Intro] Ooh, I fall apart Ooh, yeah, mmm, yeah...","On “I Fall Apart,” Post reminisces about a hea...",7.0,126.0,2640350.0,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s,[intro] ooh i fall apart ooh yeah mmm yeah [v...,"Austin Richard Post (born July 4, 1995), bette...",https://images.genius.com/1010194fa644be099aa2...,4613.0,Male,1995.0,1990s


# First Round Cleaning

In [356]:
print('kfjkfj ' * 3)

kfjkfj kfjkfj kfjkfj 


In [357]:
def clean_text(row):
    cleaned = row['lyrics'].lower()
    
    clean_dict = {"'":'',
                  ", ":" ",
                  'ain t':'aint',
                  '\u2005':'',
                  'won t':'wont',
                  'could ve':'couldve',
                  'shoul ve':'shouldve',
                  'would ve':'wouldve',
                  'i ve':'ive',
                  '-':'_',
                  'ain ':'aint ',
                  'in ':'ing ',
                  ' ing ':'ing '}
    
    for key in clean_dict:
        cleaned = cleaned.replace(key, clean_dict[key])
 
    rep_words = ['hol ','ooh ','oh ','love ','know ','yeah ','baby, ', 'gone, ', 'down, ',
                 'woah ','hey ','fight ','tryin ','skrt ', 'ma ', 'ooh yeah, ','hey, ',
                 'wild ','uh ','hillbilly ','baby ','wiggle ','fight ','want ']
    
    for rep in rep_words:
        for i in range (25):
            temp_text = rep * 3
            cleaned = cleaned.replace(temp_text,'')
            temp_text = rep * 2
            cleaned = cleaned.replace(temp_text,'')
       
    return cleaned

songs['cleaned_lyrics'] = songs.apply(clean_text, axis = 1)

In [358]:
songs[songs['artist'] == 'Justin Bieber']

,song_name,artist,lyrics,descriptionsong,accepted_annotations,contributors,pageviews,descriptionartist,images_urlsong,followers_countsong,gendersong,birth_yearsong,birth_decadesong,cleaned_lyrics,description,images_urlartist,followers_countartist,genderartist,birth_yearartist,birth_decadeartist
329,Love Yourself,Justin Bieber,[Produced by Benny Blanco] [Verse 1] For all ...,"With the help of Benny Blanco, Justin Bieber a...",9.0,201.0,3993905.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[produced by benny blanco] [verse 1] for all ...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s
330,Sorry,Justin Bieber,"[Written by Julia Michaels, Justin Tranter, an...",Capping off a year filled with public apologie...,10.0,174.0,2275309.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[written by julia michaels justing tranter and...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s
331,Yummy,Justin Bieber,"[Chorus] Yeah, you got that yummy-yum That yum...",“Yummy” is the lead single to Justin Bieber’s ...,6.0,223.0,1933936.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[chorus] yeah you got that yummy_yum that yumm...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s
332,Baby,Justin Bieber,[Produced by The-Dream and Tricky Stewart] [I...,“Baby” is about breaking up with someone you l...,1.0,172.0,1692121.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[produced by the_dream and tricky stewart] [i...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s
333,As Long As You Love Me,Justin Bieber,[Intro: Justin Bieber] As long as you love me ...,Third and final promotion single off his third...,5.0,129.0,1567192.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[intro: justing bieber] as long as you love me...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s
334,What Do You Mean?,Justin Bieber,"[Intro] [Chorus] What do you mean? Oh, oh Whe...",https://twitter.com/kanyewest/status/709070533...,6.0,113.0,1164716.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[intro] [chorus] what do you mean? when you n...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s
335,Peaches,Justin Bieber,[Chorus: Justin Bieber] I got my peaches out i...,“Peaches” is a song by Justin Bieber featuring...,4.0,150.0,1046230.0,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s,[chorus: justing bieber] i got my peaches outi...,"Justin Bieber (born March 1, 1994) is a pop, R...",https://images.genius.com/8c95afb142e16e2e053b...,3587.0,Male,1994.0,1990s


In [359]:
list(songs['lyrics'])[2]

'[Intro: Post Malone] Mm-mmm Yeah, yeah Mm-mmm Yeah (Hey)  [Chorus: Post Malone] My momma called, seen you on TV, son Said shit done changed ever since we was on I dreamed it all ever since I was young They said I wouldn\'t be nothing Now they always say, "Congratulations" (Uh, uh, uh) Worked so hard, forgot how to vacation (Uh-huh) They ain\'t never had the dedication (Uh, uh) People hatin\', say we changed and look, we made it (Uh, uh) Yeah, we made it (Uh, uh, uh)  [Verse 1: Post Malone] They was never friendly, yeah Now I\'m jumping out the Bentley, yeah And I know I sound dramatic, yeah But I know I had to have it, yeah For the money, I\'m a savage, yeah I be itching like a addict, yeah I\'m surrounded, twenty bad bitches, yeah But they didn\'t know me last year, yeah Everyone wanna act like they important (Yeah-yeah-yeah, yeah-yeah-yeah) But all that mean nothing when I saw my dough, yuh (Yeah-yeah-yeah, yeah-yeah-yeah) Everyone countin\' on me, drop the ball, yuh (Yeah-yeah-yeah

In [360]:
list(songs['cleaned_lyrics'])[2]

'[intro: post malone] mm_mmm mm_mmm yeah (hey)  [chorus: post malone] my momma called seen you on tv son said shit done changed ever since we was on i dreamed it all ever since i was young they said i wouldnt be nothing now they always say "congratulations" (uh) worked so hard forgot how to vacation (uh_huh) they aint never had the dedication (uh uh) people hating say we changed and look we made it (uh uh) yeah we made it (uh)  [verse 1: post malone] they was never friendly yeah now im jumping out the bentley yeah and i know i sound dramatic yeah but i know i had to have it yeah for the money im a savage yeah i be itching like a addict yeah im surrounded twenty bad bitches yeah but they didnt know me last year yeah everyone wanna act like they important (yeah_yeah_yeah yeah_yeah_yeah) but all that mean nothing when i saw my dough yuh (yeah_yeah_yeah yeah_yeah_yeah) everyone counting on me drop the ball yuh (yeah_yeah_yeah yeah_yeah_yeah) everything custom like im at the border if you f

In [361]:
[1,1,1,1].count(1)

4

# Building Stop Words

In [362]:
# adding artist names to stopwords
artist_names = []
for a in list(artists['artist']):
    artist_names.append(a.lower())

In [363]:
from sklearn.feature_extraction import text 
my_stop_words = ['intro','outro','chorus','verse','don','s','jason derulo','nick jonas'] + artist_names
stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

# Count Vectorizer

In [364]:
from sklearn.feature_extraction.text import CountVectorizer

word_vectorizer = CountVectorizer(ngram_range=(2,2), analyzer='word',stop_words=stop_words)
sparse_matrix = word_vectorizer.fit_transform(songs['cleaned_lyrics'])

In [365]:
frequencies = sum(sparse_matrix).toarray()[0]
df2 = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency']).reset_index()

df2.sort_values('frequency').tail(10)

,index,frequency
7025,bitch im,158
96801,yeah im,187
49552,like im,190
52717,love love,198
45783,know im,204
45813,know know,205
25706,feel like,216
97299,yeah yeah,297
21182,dont know,327
21382,dont wanna,360


In [366]:
songs.shape

(923, 20)

# Analysis By Artist Gender

In [344]:
female = songs[songs['gender'] == 'Female']
male = songs[songs['gender'] == 'Male']

In [345]:
female.shape

(231, 14)

In [346]:
male.shape

(595, 14)

#### female

In [347]:
word_vectorizer = CountVectorizer(ngram_range=(5,5), analyzer='word',stop_words=stop_words)
sparse_matrix = word_vectorizer.fit_transform(female['cleaned_lyrics'])

In [348]:
frequencies = sum(sparse_matrix).toarray()[0]
f_count = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency']).reset_index()

f_count.sort_values('frequency').tail(10)

,index,frequency
1893,beezing trap bee beezing trap,16
17066,love love love love love,17
9263,girl girl girl girl girl,17
12484,im cow bitch im cow,18
2325,bitch im cow bitch im,18
20046,oh_oh oh_oh oh_oh oh_oh oh_oh,18
26186,thats type nigga thats type,21
18588,moo moo moo moo moo,23
13066,im trying im trying im,30
27519,trying im trying im trying,36


#### male

In [349]:
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word',stop_words=stop_words)
sparse_matrix = word_vectorizer.fit_transform(male['cleaned_lyrics'])

In [350]:
frequencies = sum(sparse_matrix).toarray()[0]
m_count = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency']).reset_index()

m_count.sort_values('frequency').tail(100)

,index,frequency
82896,tell tell tell,15
85063,thunder th_th_thunder thunder,15
29934,god ing bank,15
53427,lot times did,15
93599,wit da hoodie,16
93804,woke niggas talking,16
93803,woke niggas sounding,16
57247,mind mind mind,16
57183,mind got murder,16
95635,yeah bitch yeah,16


# Analysis By Artist Age

In [351]:
Seven = songs[songs['birth_decade'] == '1970s']
Eight = songs[songs['birth_decade'] == '1980s']
Nine = songs[songs['birth_decade'] == '1990s']
Two_Thousands = songs[songs['birth_decade'] == '2000s']